In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot
from datetime import datetime, timedelta

from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelBinarizer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasClassifier

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) #to supress import warnings

Using TensorFlow backend.


# Preprocessing Data

In [ ]:
#COLUMNS = ['station','date','feature', 'value', 'measurement','quality', 'source', 'hour']
COLUMNS_test = ['station','date']

In [ ]:
# load data
df_train = pd.read_csv('../data/export_features_loc_MAX.csv', index_col=0, low_memory=False)

In [ ]:
df_train['date'] = pd.to_datetime(df_train['date'], format='%Y%m%d', errors='ignore')
df_train.head()

In [ ]:
# Do you want to use past days as predictor?
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [ ]:
#get year and date as features
df = df_train
df = df_train_yd.drop(columns='date')
df['year'] = df_train['date'].map(lambda x: x.year)
df['day'] = df_train['date'].map(lambda x: x.timetuple().tm_yday)
df.head(20)

In [ ]:
# export data to reduce preproccesing duration
#df_train_yd.to_csv('../data/tmp/export_LSMT_MAX_yd.csv')

### Getting the data ready for LMST

In [2]:
# importing the preprocessed data for a quicker start
df = pd.read_csv('../data/tmp/export_LSMT_MAX_yd.csv', index_col=0, low_memory=False)

In [3]:
# pick random stations for test and training
seed = 93598357
np.random.seed(seed)
stations = df.station.unique()
np.random.shuffle(stations)
stations_shuffled = stations
stations_train = stations_shuffled[:int(np.round(len(stations)/4))]
stations_holdout14 = stations_shuffled[int(np.round(len(stations)/4)):int(np.round(len(stations)/2))]
stations_holdout15 = stations_shuffled[int(np.round(len(stations)/2)):int(np.round(len(stations)/4*3))]
stations_holdout16 = stations_shuffled[int(np.round(len(stations)/4*3)):]

df_train_test = df[df['station'].isin(stations_train)]
df_14 = df[df['station'].isin(stations_holdout14)]
df_15 = df[df['station'].isin(stations_holdout15)]
df_16 = df[df['station'].isin(stations_holdout16)]

In [4]:
print(len(df_train_test), len(df_14), len(df_15), len(df_16))

4639908 4680661 4705872 4657383


In [13]:
#divide test and training to test effective of model to different timeframe (start of 2017)
training_years = [2014,2015,2016]
testing_days = list(range(90))

df_train = df_train_test[df_train_test['year'].isin(training_years)]
df_test = df_train_test[~df_train_test['year'].isin(testing_days)]
df_test = df_test[df_test['day'].isin(testing_days)]
print(df_train.shape,df_test.shape)

training_years = [2017,2015,2016]
df_train14 = df_14[df_14['year'].isin(training_years)]
df_test14 = df_14[~df_14['year'].isin(testing_days)]
df_test14 = df_test14[df_test14['day'].isin(testing_days)]
print(df_train.shape,df_test.shape)

training_years = [2017,2014,2016]
df_train15 = df_15[df_15['year'].isin(training_years)]
df_test15 = df_15[~df_15['year'].isin(testing_days)]
df_test15 = df_test15[df_test15['day'].isin(testing_days)]
print(df_train.shape,df_test.shape)

training_years = [2017,2015,2014]
df_train16 = df_16[df_16['year'].isin(training_years)]
df_test16 = df_16[~df_16['year'].isin(testing_days)]
df_test16 = df_test16[df_test16['day'].isin(testing_days)]
print(df_train.shape,df_test.shape)


(3534546, 7) (1143123, 7)
(3534546, 7) (1143123, 7)
(3534546, 7) (1143123, 7)
(3534546, 7) (1143123, 7)


In [17]:
#export model data for the cloud training
df_train.to_csv('../data/aws_train')
df_test.to_csv('../data/aws_test')

In [18]:
df_train14.to_csv('../data/aws_train14')
df_test14.to_csv('../data/aws_test14')

In [19]:
df_train15.to_csv('../data/aws_train15')
df_test15.to_csv('../data/aws_test15')

In [20]:
df_train16.to_csv('../data/aws_train16')
df_test16.to_csv('../data/aws_test16')

In [ ]:
#seperate target from features
df_X_train_raw = df_train.drop(columns='TMIN')
df_X_test_raw = df_test.drop(columns='TMIN')
sy_train = df_train['TMIN'].values
sy_test = df_test['TMIN'].values
y_train_raw = sy_train.reshape(-1,1)
y_test_raw = sy_test.reshape(-1,1)

In [ ]:
df_X_train_raw.shape

In [ ]:
# int encode stations
#LB = LabelBinarizer()
#df_X['station'] = LB.fit_transform(df_X[['station']])
df_X_train_red = df_X_train_raw.drop(columns='station')
df_X_test_red = df_X_test_raw.drop(columns='station')

In [ ]:
#X_dict = df_X.to_dict('records')
#vec = DictVectorizer()
#X = vec.fit_transform(X_dict).toarray()
#X_dummies = pd.get_dummies(df_X)
#X = X_dummies.to_dict('records')

In [ ]:
# normalize features
X_train_raw = df_X_train_red.values
X_test_raw = df_X_test_red.values
X_train_raw = X_train_raw.astype('float32')
X_test_raw = X_test_raw.astype('float32')
y_train_raw = y_train_raw.astype('float32')
y_test_raw = y_test_raw.astype('float32')

scaler = MinMaxScaler(feature_range=(0, 1))                             
train_X = scaler.fit_transform(X_train_raw)
test_X = scaler.fit_transform(X_test_raw)
train_y = scaler.fit_transform(y_train_raw)
test_y = scaler.fit_transform(y_test_raw)

In [ ]:
print(train_X.shape,train_y.shape,test_X.shape,test_y.shape)

In [ ]:
# specify the number of lag days
n_days = 0
n_features = 1
# frame as supervised learning
#reframed = scaled #series_to_supervised(scaled, n_days, 1)

In [ ]:
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((1792641,1,5))
test_X = test_X.reshape((514252,1,5))

# LSTMs

In [ ]:
# design network
learning_rate = 0.2
epochs = 50
decay_rate = learning_rate / epochs

model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1],train_X.shape[2])))
model.add(LSTM(10))
model.add(Dense(1))
adam = Adam(lr = learning_rate, decay=decay_rate)
model.compile(loss='mae', optimizer=adam)
# fit network
history = model.fit(train_X, train_y, epochs=epochs, batch_size=200, validation_data=(test_X, test_y), verbose=2, shuffle=False)
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
# make a prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
# invert scaling for forecast
inv_yhat = np.concatenate((yhat, test_X[:, 1:]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = np.concatenate((test_y, test_X[:, 1:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]
# calculate RMSE
rmse = np.sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

In [ ]:
# grid search for optimal parameters: batch and epochs
def create_model():
    model = Sequential()
    model.add(LSTM(50, input_shape=(train_X.shape[1],train_X.shape[2])))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    # Compile model
    model.compile(loss='mae', optimizer='adam', metrics=['accuracy'])
    return model

np.random.seed(seed)
# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 20, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(train_X, train_y).score(X_test, y_test)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
# grid search for optimal parameters: dropout rate
def create_model():
    model = Sequential()
    model.add(LSTM(50, input_shape=(train_X.shape[1],train_X.shape[2])))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    # Compile model
    model.compile(loss='mae', optimizer='adam', metrics=['accuracy'])
    return model

np.random.seed(seed)
# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
batch_size = 
epochs = 
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
param_grid = dict(dropout_rate=dropout_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(train_X, train_y).score(X_test, y_test)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
# make a prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
# invert scaling for forecast
inv_yhat = np.concatenate((yhat, test_X[:, 1:]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = np.concatenate((test_y, test_X[:, 1:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]
# calculate RMSE
rmse = np.sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

In [ ]:
# import kaggle scoring data
df_score = pd.read_csv('../data/2018_test.csv', header=None, names=COLUMNS_test, low_memory=False)